In [1]:
!pip install matplotlib_inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [119]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')
import scipy.stats as ss
from scipy import optimize
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint
import pandas as pd

**Задача 1**


In [118]:
data = np.array([(576, 3.39), (635, 3.30), (558, 2.81), (578, 3.03), (666, 3.44),
                 (580, 3.07), (555, 3.0), (661, 3.43), (651, 3.36), (605, 3.13),
                 (653, 3.12), (575, 2.74), (545, 2.76), (572, 2.88), (594, 2.96)])

# g(p*,p) = \sqrt(n)(p* - p) / (1 - p^2)


p_star = np.corrcoef(data.T)[0,1]
alpha = 0.95
z_L = ss.norm().ppf((1-alpha)/2)
z_R = ss.norm().ppf((1+alpha)/2)
n = len(data)

def g(p):
  return np.sqrt(n) * (p_star - p) / (1 - p**2)

solution_R = optimize.root_scalar(lambda p: g(p) - z_L, bracket = [-0.999, 0.999])
solution_L = optimize.root_scalar(lambda p: g(p) - z_R, bracket = [-0.999, 0.999])

solution_L.root, solution_R.root

(0.3231645687328118, 0.8855673805132537)

**Задача 2**

In [120]:
with open('casino.txt', 'r') as file:
    data = []
    for row in file.readlines():
        data.append(list(map(int, row.strip().split())))


def fun(x):
    p_win, p_go_after_win, p_go_after_lose = x
    p_lose = 1 - p_win
    res = 0
    for d in data:
      d_ = np.array(d)
      ones = d_[:-1].sum()
      zeros = (d_[:-1] == 0).sum()
      res_ = ones * np.log(p_win * (1 - p_go_after_win)) + zeros * np.log(p_lose * (1 - p_go_after_lose))
      if d[-1] == 1:
        res_ += np.log(p_win * p_go_after_win)
      else:
        res_ += np.log(p_lose * p_go_after_lose)

      res += res_

    return -res

bounds=[(0.0001, .999999) for _ in range(3)]

res = minimize(fun, x0=np.array([1/2,1/2,1/2]), bounds=bounds)
res.x

array([0.48417593, 0.34059625, 0.75672812])

**Задача 3**

In [122]:
with open('casino.txt', 'r') as file:
    data = []
    for row in file.readlines():
        data.append(list(map(int, row.strip().split())))

np.random.seed(7)

def f(x, data):
    p_win, p_go_after_win, p_go_after_lose = x
    p_lose = 1 - p_win
    res = 0
    for d in data:
      d_ = np.array(d)
      ones = d_[:-1].sum()
      zeros = (d_[:-1] == 0).sum()
      res_ = ones * np.log(p_win * (1 - p_go_after_win)) + zeros * np.log(p_lose * (1 - p_go_after_lose))
      if d[-1] == 1:
        res_ += np.log(p_win * p_go_after_win)
      else:
        res_ += np.log(p_lose * p_go_after_lose)

      res += res_

    return -res


data = np.array(data, dtype=object)
bounds=[(0.0001, .999999) for _ in range(3)]

N = 100
n = 1000
idx = np.random.choice(n, size = (N, n))
Data = data[idx]

phi_b = []
for my_data in Data:
  res = minimize(lambda x: f(x, my_data), x0=np.array([1/2,1/2,1/2]), bounds=bounds)
  phi_b.append(res.x[2] - res.x[1])

phi_b = np.array(phi_b)
alpha = 0.95
np.quantile(phi_b, (1 - alpha) / 2), np.quantile(phi_b, (1 + alpha) / 2)


(0.37850622188741634, 0.45990550288201604)

**Задача 4**

In [123]:
# D_X = 24\epsilon + 1
# 4p(0)^2 = 4(1 - (4/5)\epsilon)^2(1/2pi)

def relation(e):
  return (2/np.pi) * (24* e + 1) * (1 - 4 * e / 5) ** 2 - 1


solution = optimize.root_scalar(relation, bracket = [-1, 1/2])
solution.root

0.02666780668662032

**Задача 5**

In [127]:
np.random.seed(42)

data =pd.read_csv('Advertising.csv')
data = data[["TV","sales"]]
data = data.to_numpy()
N = 10000
n = 200
idx = np.random.choice(n, size = (N, n))
samples = data[idx]

def L_2(x, data):
  c, b = x
  tv = data.T[0]
  sales = data.T[1]
  return ((sales - b * tv - c)**2).sum()

phi_b = []
bounds = [(None, None) for _ in range(2)]

for my_data in samples:
  res = minimize(lambda x: L_2(x, my_data), x0=np.zeros(2), bounds=bounds)
  phi_b.append(np.array([res.x[0], res.x[1]]))

phi_b = np.array(phi_b)
np.corrcoef(phi_b.T)[0,1]

-0.8424340179959471